In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import feature_extraction,model_selection,preprocessing


In [3]:
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')

In [6]:
train_df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [22]:
import nltk
from nltk.corpus import stopwords,wordnet
from nltk.tokenize import word_tokenize

In [24]:
from nltk.stem.porter import PorterStemmer
stemmer=PorterStemmer()
stop_words=set(stopwords.words('english'))
stop_words=stop_words.remove('not')


In [25]:
import regex as re
text=re.sub(r'[^a-zA-Z\s]', '', train_df['text'][6])
print(train_df['text'][6])
print(text)

#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas
flood disaster Heavy rain causes flash flooding of streets in Manitou Colorado Springs areas


In [61]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
if 'not' in stop_words:  
    stop_words.remove('not')

def preprocess(x):
    corpus = []
    for t in x:
        t = re.sub(r'[^a-zA-Z\s]', '', t)
        t = t.lower()
        tokens = word_tokenize(t)
        filtered_tokens = [stemmer.stem(token) for token in tokens if token not in stop_words]
        corpus.append(" ".join(filtered_tokens))
    return corpus   
        

In [62]:
train_df['clean_text']=preprocess(train_df['text'].values)

In [63]:
test_df['clean_text']=preprocess(test_df['text'].values)

In [67]:
train_df['clean_text'].head

<bound method NDFrame.head of 0               deed reason earthquak may allah forgiv us
1                    forest fire near la rong sask canada
2       resid ask shelter place notifi offic evacu she...
3             peopl receiv wildfir evacu order california
4       got sent photo rubi alaska smoke wildfir pour ...
                              ...                        
7608    two giant crane hold bridg collaps nearbi home...
7609    ariaahrari thetawniest control wild fire calif...
7610                utckm volcano hawaii httptcozdtoydebj
7611    polic investig ebik collid car littl portug eb...
7612    latest home raze northern california wildfir a...
Name: clean_text, Length: 7613, dtype: object>

In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=500)  
X = vectorizer.fit_transform(train_df["clean_text"]).toarray()
y = train_df["target"]

In [69]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [71]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
models = {
    "Naive Bayes": MultinomialNB(),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "SVM": LinearSVC()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(f"{name} Results:")
    print("Accuracy:", accuracy_score(y_test, preds))
    print("Precision:", precision_score(y_test, preds))
    print("Recall:", recall_score(y_test, preds))
    print("F1:", f1_score(y_test, preds))
    print()

Naive Bayes Results:
Accuracy: 0.7662508207485227
Precision: 0.7707948243992606
Recall: 0.6425269645608629
F1: 0.7008403361344537

Logistic Regression Results:
Accuracy: 0.7767564018384767
Precision: 0.768695652173913
Recall: 0.6810477657935285
F1: 0.7222222222222222

SVM Results:
Accuracy: 0.778069599474721
Precision: 0.7604690117252931
Recall: 0.699537750385208
F1: 0.7287319422150883



In [73]:
from sklearn.metrics import classification_report, confusion_matrix
best_model = MultinomialNB().fit(X_train, y_train)
y_pred = best_model.predict(X_test)

print(classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.86      0.81       874
           1       0.77      0.64      0.70       649

    accuracy                           0.77      1523
   macro avg       0.77      0.75      0.75      1523
weighted avg       0.77      0.77      0.76      1523

Confusion Matrix:
 [[750 124]
 [232 417]]
